In [1]:
import numpy as np
import pandas as pd
from kmodes.kprototypes import KPrototypes
from patsy import dmatrices

import statsmodels.api as sm
from sklearn.metrics import r2_score 

import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime
from calendar import monthrange
import calendar


In [2]:
#https://data.iledefrance-mobilites.fr/explore/dataset/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-sem/table/
df = pd.read_csv('validations.csv', sep = ';')

##  - Quelles sont les 20 premières stations en terme de validations ?
##  - Proposez une illustration graphique de ce classement.


d'après la documentation, la mention « Moins de 5 » dans la colonne NB_VALD  est indiquée pour les volumes inférieurs à 5 non nuls.

- On remplace les valeurs de NB_VALD 'Moins de 5' par 3

In [3]:
df.loc[(df.NB_VALD == 'Moins de 5'),'NB_VALD']= 3
df['NB_VALD'] = df.NB_VALD.astype('int')
df['JOUR'] = pd.to_datetime(df.JOUR)


In [4]:
print('Notre jeu de données se compose de',df.shape[0], 'validations définis dans la période entre ', df.JOUR.min(), 'et', 
      df.JOUR.max(),' pour ', len(df.LIBELLE_ARRET.unique()), 'stations')

Notre jeu de données se compose de 940549 validations définis dans la période entre  2019-01-01 00:00:00 et 2019-06-30 00:00:00  pour  732 stations


In [5]:
#groupby Station
grouped_stations = df.groupby('LIBELLE_ARRET')['NB_VALD'].sum()
grouped_stations = grouped_stations.to_frame()
grouped_stations = grouped_stations.reset_index()

In [6]:
grouped_stations = grouped_stations.sort_values(['NB_VALD'],ascending=False)[1:21]

In [10]:
fig = px.bar(grouped_stations, y='NB_VALD', x='LIBELLE_ARRET', text='NB_VALD')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout( title = 'Illustration graphique des 20 premières stations en terme de validations')
fig.show()

- les stations 'Gare du Nord' et 'Gare de Lyon' représentent les stations avec le plus grand nombre de validations dans la période étudiée 
avec 29M et 27M de validations respectivement, cela est justifié par la nature de ces stations qui représentent un hub de correspendance pour
les usagers des différents lignes de transport.

## - Proposez une classification des arrêts en vous basant sur le nombre de validations par catégorie de titre ?



In [11]:
# on regroupe pour chaque station le nombre de validaions pour chaque catégorie de titre.
gp = df.groupby(['LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum()
gp = gp.to_frame()
gp = gp.reset_index()
gp

,LIBELLE_ARRET,CATEGORIE_TITRE,NB_VALD
0,ABBESSES,?,3539
1,ABBESSES,AMETHYSTE,17217
2,ABBESSES,AUTRE TITRE,903
3,ABBESSES,FGT,14654
4,ABBESSES,IMAGINE R,103101
...,...,...,...
6147,YERRES,FGT,22323
6148,YERRES,IMAGINE R,117962
6149,YERRES,NAVIGO,555607
6150,YERRES,NAVIGO JOUR,501


In [12]:
mark_array = gp[['CATEGORIE_TITRE', 'NB_VALD']].values


In [14]:
#Choosing optimal K value
cost = []
X = mark_array
for num_clusters in list(range(2,7)):
    kproto = KPrototypes(n_clusters=num_clusters, init='Huang', random_state=42,n_jobs=2,max_iter=15,n_init=50,verbose =2) 
    kproto.fit_predict(X, categorical=[0])
    cost.append(kproto.cost_)

fig = px.line(x =list(range(2,7)), y=cost, title='Visualisation K-Elbow ',
              labels=dict(x="K (nombre de clusters)", y="Distorsion"))
fig.show()

Best run was number 2
Best run was number 1
Best run was number 1
Best run was number 12
Best run was number 50


- d'après la visualisation de la méthode K-Elbow, On va choisir K = 3. 

In [15]:
kproto = KPrototypes(n_clusters=3, init='Huang',max_iter=20)#n_jobs=2 # random_state=42n_init=50
kproto.fit_predict(mark_array, categorical=[0])

array([2, 2, 2, ..., 2, 2, 2], dtype=uint16)

In [16]:
gp['clusters']= kproto.labels_

In [17]:
gp.groupby('clusters').describe()

NB_VALD                                                    \
           count          mean           std        min        25%   
clusters                                                             
0           10.0  1.416072e+07  5.631548e+06  8343353.0  9981954.0   
1          287.0  1.626802e+06  8.955298e+05   845190.0  1047987.5   
2         5855.0  6.349560e+04  1.345822e+05        3.0      586.0   

                                               
                 50%          75%         max  
clusters                                       
0         12642386.5  18633467.75  24635570.0  
1          1346331.0   1866051.00   7154308.0  
2             7837.0     51936.00    836890.0

- Le tableau résume les caractéristiques des différents clusters

In [42]:
print("En analysant le cluster 0, On remarque que les validations sont issues de",len(gp[gp['clusters']==0].LIBELLE_ARRET.unique())," \
stations différentes, à l'exception de la station LA DEFENSE GRANDE ARCHE, ces stations représentent les 20 stations avec le plus grand nombre   \
de validations qu'on a affichées en haut. Le cluster 0 regroupe ainsi les stations avec\
les plus grandes affluences de passagers dans la période étudiée. Ces validations varient entre  8343353.0 et 24635570.0 avec une moyenne de \
1416072 validations et proviennent seulement du pass Navigo. Nous remarquons ainsi \
que le cluster 0 contient seulement 10 élements avec une variance importante, Il est également possible de regrouper le cluster0 et le cluster 1\n")                                                                                                 

En analysant le cluster 0, On remarque que les validations sont issues de 10  stations différentes, à l'exception de la station LA DEFENSE GRANDE ARCHE, ces stations représentent les 20 stations avec le plus grand nombre   de validations qu'on a affichées en haut. Le cluster 0 regroupe ainsi les stations avecles plus grandes affluences de passagers dans la période étudiée. Ces validations varient entre  8343353.0 et 24635570.0 avec une moyenne de 1416072 validations et proviennent seulement du pass Navigo. Nous remarquons ainsi que le cluster 0 contient seulement 10 élements avec une variance importante, Il est également possible de regrouper le cluster0 et le cluster 1



In [43]:
print("Le cluster 1 contient (287 validations différentes selon la categorie de validation pour chaque station).Le nombre de validation varient  \
entre 845190.0 et 7154308.0 validations avec une moyenne de 1626802. Ces validations proviennet de    \
", len(gp[gp['clusters']==1].LIBELLE_ARRET.unique()),"stations différentes, et sont issus des différents moyens de validations disponibles. \
le cluster 1 regroupe ainsi les stations avec l'affluence de passagers moyenne sur la période étudiée. \n" )
      

le cluster 1 contient (287 validations différentes selon la categorie de validation pour chaque station).Le nombre de validation varient  entre 845190.0 et 7154308.0 validations avec une moyenne de 1626802. Ces validations proviennet de     269 stations différentes, et sont issus des différents moyens de validations disponibles. le cluster 1 regroupe ainsi les stations avec l'affluence de passagers moyenne sur la période étudiée. 



In [44]:
print("Il est bien claire que le cluster 2 contient le plus grand nombre d'individus (5855 validations différentes selon la categorie de validation \
pour chaque  station).Le nombre de validations varient entre 3 et 836890 validations avec une moyenne de 63495.Ces validations proviennent de\
",len(gp[gp['clusters']==2].LIBELLE_ARRET.unique()),"stations différentes, et sont issus des différents moyens de validations disponibles. \
le cluster 2 regroupe ainsi les stations avec l'affluence de passagers la plus faible sur la période étudiée.")
      

Il est bien claire que le cluster 2 contient le plus grand nombre d'individus (5855 validations différentes selon la categorie de validation pour chaque  station).Le nombre de validations varient entre 3 et 836890 validations avec une moyenne de 63495.Ces validations proviennent de 732 stations différentes, et sont issus des différents moyens de validations disponibles. le cluster 2 regroupe ainsi les stations avec l'affluence de passagers la plus faible sur la période étudiée.


## Quelles prédictions pouvez vous faire pour le nombre de validations dans les stations "OLYMPIADES" et "M. MONTROUGE" sur la plage du 24 au 30 juin ?#

In [45]:
#selectionner les stsations concernées.
dataset = df.loc[df['LIBELLE_ARRET'].isin(['OLYMPIADES', 'M. MONTROUGE'])].sort_values(['JOUR'])
dataset = dataset.groupby(['JOUR','LIBELLE_ARRET'],as_index=False)['NB_VALD'].sum()
dataset

,JOUR,LIBELLE_ARRET,NB_VALD
0,2019-01-01,M. MONTROUGE,2687
1,2019-01-01,OLYMPIADES,3764
2,2019-01-02,M. MONTROUGE,13436
3,2019-01-02,OLYMPIADES,14072
4,2019-01-03,M. MONTROUGE,15744
...,...,...,...
357,2019-06-28,OLYMPIADES,20413
358,2019-06-29,M. MONTROUGE,9910
359,2019-06-29,OLYMPIADES,12147
360,2019-06-30,M. MONTROUGE,6062


In [46]:
dataset['JOUR'] = pd.to_datetime(dataset['JOUR'], format='%Y-%m-%d')
#adding extra variables weekday
dataset['weekday'] = dataset['JOUR'].dt.day_name()


In [47]:
#Check No missing data
dataset.isnull().values.any()


False

- Nous savons que l'affluence de passagers dans les stations peut variées selon les journées de travails ou les jours de vacances, J'ai ainsi
ajouté les différents jours fériés nationaux dans la période étudié.
source: https://calendrierfrance.fr/jours-feries-2019/

In [48]:
df_holidays = pd.read_excel('holidays.xlsx', sep = ';')
df_holidays['Date'] = pd.to_datetime(df_holidays['Date'], format = '%d %b %Y') 
df_holidays = df_holidays[df_holidays['Date'].dt.month < 7]
df_holidays = df_holidays.rename(columns={'Jour férié':'holiday'})

In [49]:
new_data = dataset.merge(df_holidays, left_on='JOUR', right_on='Date', how='left')[['JOUR','LIBELLE_ARRET', 'NB_VALD', 'weekday', 'holiday']]
new_data['holiday'] = new_data['holiday'].fillna('False')
new_data['holiday'] = new_data['holiday'].apply(lambda x: True if x!='False' else False)

In [50]:
new_data['workingday'] = new_data['weekday'].isin(['Saturday', 'Sunday']) | new_data['holiday'] == True
new_data['day'] = new_data['JOUR'].dt.day
new_data['month'] = new_data['JOUR'].dt.month

In [51]:
def encode(month, col, max_val):
    "la fonction permet d encoder cycliquement la variables catégorielle"
    global new_data
    new_data.loc[(new_data['month']==month),col + '_sin'] = np.sin(2 * np.pi * new_data[new_data['month']==month][col] / max_val)
    new_data.loc[(new_data['month']==month),col + '_cos'] = np.cos(2 * np.pi * new_data[new_data['month']==month][col] / max_val)
    

In [52]:
months = list(new_data['month'].unique())
for month in months:
    encode(month, 'day', monthrange(2019, month)[1])

In [54]:
new_data['JOUR'] = pd.to_datetime(new_data.JOUR)
new_data['LIBELLE_ARRET'] = new_data.LIBELLE_ARRET.astype('category')
new_data['weekday'] = new_data.weekday.astype('category')
new_data['holiday'] = new_data.holiday.astype('category')
new_data['workingday'] = new_data.workingday.astype('category')
new_data['month'] = new_data.month.astype('category')
new_data['day'] = new_data.day.astype('category')
new_data

,JOUR,LIBELLE_ARRET,NB_VALD,weekday,holiday,workingday,day,month,day_sin,day_cos
0,2019-01-01,M. MONTROUGE,2687,Tuesday,True,True,1,1,2.012985e-01,0.979530
1,2019-01-01,OLYMPIADES,3764,Tuesday,True,True,1,1,2.012985e-01,0.979530
2,2019-01-02,M. MONTROUGE,13436,Wednesday,False,False,2,1,3.943559e-01,0.918958
3,2019-01-02,OLYMPIADES,14072,Wednesday,False,False,2,1,3.943559e-01,0.918958
4,2019-01-03,M. MONTROUGE,15744,Thursday,False,False,3,1,5.712682e-01,0.820763
...,...,...,...,...,...,...,...,...,...,...
357,2019-06-28,OLYMPIADES,20413,Friday,False,False,28,6,-4.067366e-01,0.913545
358,2019-06-29,M. MONTROUGE,9910,Saturday,False,True,29,6,-2.079117e-01,0.978148
359,2019-06-29,OLYMPIADES,12147,Saturday,False,True,29,6,-2.079117e-01,0.978148
360,2019-06-30,M. MONTROUGE,6062,Sunday,False,True,30,6,-1.133108e-15,1.000000


### Construction données d'apprentissage avant 24 Juin et les données de test après 24 Juin

In [55]:
df_train = new_data.iloc[0:348, 1:]
df_test = new_data.iloc[348:,1:]
test_dates = pd.DataFrame(new_data.iloc[348:,0:3])
test_dates['JOUR'] = test_dates['JOUR'].dt.strftime('%Y-%m-%d')

### Tester différents variables et comparer plusieurs modèles par rapport au BIC

In [57]:
model1 = """NB_VALD ~ LIBELLE_ARRET  + weekday + holiday + workingday + month + day_cos + day_sin"""
model2 = """NB_VALD ~ LIBELLE_ARRET  + weekday + workingday + month + day_cos + day_sin"""
model3 = """NB_VALD ~ LIBELLE_ARRET  + weekday + holiday + workingday + month + day"""

### La variable à prédire NB_VALD est une variable de comptage (toujours positive), nous choissisons ainsi un  modèle classique de poisson. 

In [58]:
# tester trois différents modèles 
models = [model1, model2, model3]
bic_scores = []
for model in models:
    y_train, X_train = dmatrices(model, df_train, return_type='dataframe')
    y_test, X_test = dmatrices(model, df_test, return_type='dataframe')
    poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
    bic_scores.append(poisson_training_results.bic)


In [59]:
bic_scores

[70203.80711647471, 70203.80711647467, 67433.95959545329]

### En comparant les différents models en terme de BIC, On remarque que le troisème modèle est meilleur. 

In [60]:
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                NB_VALD   No. Observations:                  348
Model:                            GLM   Df Residuals:                      304
Model Family:                 Poisson   Df Model:                           43
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -36609.
Date:                Mon, 26 Oct 2020   Deviance:                       69213.
Time:                        15:02:04   Pearson chi2:                 6.75e+04
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [61]:
poisson_predictions = poisson_training_results.get_prediction(X_test)

#arrondissement des valeurs prédites par le model.
test_dates['Predicted_NB_VALD'] = round(poisson_predictions.summary_frame()['mean']).values

In [62]:
test_dates

,JOUR,LIBELLE_ARRET,NB_VALD,Predicted_NB_VALD
348,2019-06-24,M. MONTROUGE,19779,19158.0
349,2019-06-24,OLYMPIADES,19622,21136.0
350,2019-06-25,M. MONTROUGE,21470,19276.0
351,2019-06-25,OLYMPIADES,20171,21266.0
352,2019-06-26,M. MONTROUGE,20526,19631.0
353,2019-06-26,OLYMPIADES,19905,21658.0
354,2019-06-27,M. MONTROUGE,20899,19725.0
355,2019-06-27,OLYMPIADES,20030,21762.0
356,2019-06-28,M. MONTROUGE,20536,19309.0
357,2019-06-28,OLYMPIADES,20413,21303.0


## Visualisation selon les validations observées et prédites

In [70]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_dates.JOUR.values, y = test_dates.Predicted_NB_VALD.values,
                    mode='lines+markers',
                    name='Predicted validations'))
fig.add_trace(go.Scatter(x = test_dates.JOUR.values, y = test_dates.NB_VALD.values,
                    mode='lines+markers',
                    name='Actual validations'))

fig.update_layout(
    title="le nombre de validations observés et predits sur la plage du 24 au 30 juin",
    xaxis_title="Jour",
    yaxis_title="Nombre de validations",
)

fig.show()



### Visualisation selon la station

In [71]:
fig = go.Figure()

data1 = test_dates[ test_dates['LIBELLE_ARRET'] == 'M. MONTROUGE']
data2 = test_dates[ test_dates['LIBELLE_ARRET'] == 'OLYMPIADES']

# Add traces#test_dates
fig.add_trace(go.Scatter(x = data1.JOUR.values, y = data1.Predicted_NB_VALD.values,
                    mode='lines+markers',
                    name='MONTROUGE validations'))
fig.add_trace(go.Scatter(x = data2.JOUR.values, y = data2.NB_VALD.values,
                    mode='lines+markers',
                    name='OLYMPIADES validations'))

fig.update_layout(
    title="le nombre de validations predits sur la plage du 24 au 30 juin pour chaque station",
    xaxis_title="Jour",
    yaxis_title="Nombre de validations",
)

fig.show()



In [72]:
r2 = r2_score(test_dates.NB_VALD.values, test_dates.Predicted_NB_VALD.values) 
print('R2 score for model3 : ', r2)

R2 score for model3 :  0.9384857237507696
